<a href="https://colab.research.google.com/github/tensorflow-project/FineTuning/blob/main/two_concepts/training_two_concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tensorflow-project/FineTuning

Cloning into 'FineTuning'...
remote: Enumerating objects: 1488, done.
remote: Counting objects: 100% (451/451), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 1488 (delta 328), reused 352 (delta 247), pack-reused 1037
Receiving objects: 100% (1488/1488), 92.57 MiB | 17.70 MiB/s, done.
Resolving deltas: 100% (1047/1047), done.


In [2]:
import sys
import os
import numpy as np
from google.colab import drive
import keras
import tensorflow as tf

### agree to mounting your Google Drive
drive.mount("/content/drive")

py_file_location = "/content/FineTuning/two_concepts"
sys.path.append(os.path.abspath(py_file_location))
py_file_location = "/content/FineTuning/models"
sys.path.append(os.path.abspath(py_file_location))

import textual_inversion_two_concepts as txt

Mounted at /content/drive
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
### create an instance of the Stable Diffusion model
stable_diffusion = txt.StableDiffusion()

### our new concept which is later inserted in the different prompts (for training and image generation)
placeholder_token_broccoli = "<my-broccoli-token>"
placeholder_token_emoji = "<my-emoji-token>"
placeholder_token_combined = "<my-broccoli-emoji-token>"

### create a dataset consisting of broccoli stickers prompts
broccoli_ds = txt.assemble_dataset(
    urls = [
        "https://i.imgur.com/9zAwPyt.jpg",
        "https://i.imgur.com/qCNFRl4.jpg",
        "https://i.imgur.com/kPH9XIh.jpg",
        "https://i.imgur.com/qy1k0QK.jpg",
    ],
    prompts = [
        "a photo of a happy {}",
        "a photo of {}",
        "a photo of one {}",
        "a photo of a nice {}",
        "a good photo of a {}",
        "a photo of the nice {}",
        "a photo of a cool {}",
        "a rendition of the {}",
        "a nice sticker of a {}",
        "a sticker of a {}",
        "a sticker of a happy {}",
        "a sticker of a lucky {}",
        "a sticker of a lovely {}",
        "a sticker of a {} in a positive mood",
        "a pixar chracter of a satisfied {}",
        "a disney character of a positive {}",
        "a sticker of a delighted {}",
        "a sticker of a joyful {}",
        "a sticker of a cheerful {}",
        "a drawing of a glad {}",
        "a sticker of a merry {}",
        "a sticker of a pleased {}",
    ],
    placeholder_token = placeholder_token_broccoli, 
    stable_diffusion = stable_diffusion
)  

### create a dataset consisting of happy emojis and happy prompts
emoji_ds = txt.assemble_dataset(
    urls = [
        "https://i.imgur.com/BLLMggR.png",
        "https://i.imgur.com/PPQ2UtM.png",
        "https://i.imgur.com/6je73G3.png",
    ],
    prompts = [
        "a photo of a happy {}",
        "a photo of {}",
        "a photo of one {}",
        "a photo of a nice {}",
        "a good photo of a {}",
        "a photo of the nice {}",
        "a photo of a cool {}",
        "a rendition of the {}",
        "a nice emoji of a {}",
        "an emoji of a {}",
        "an emoji of a happy {}",
        "an emoji of a lucky {}",
        "an emoji of a lovely {}",
        "an emoji of a {} in a positive mood",
        "an emoji chracter of a satisfied {}",
        "an emoji character of a positive {}",
        "an emoji of a delighted {}",
        "an emoji of a joyful {}",
        "an emoji of a cheerful {}",
        "an emoji of a glad {}",
        "an emoji of a merry {}",
        "an emoji of a pleased {}",
    ],
    placeholder_token = placeholder_token_emoji,
    stable_diffusion = stable_diffusion
)

### concatenate the different datasets
train_ds = emoji_ds.concatenate(broccoli_ds)
train_ds = train_ds.batch(1).shuffle(
    train_ds.cardinality(), reshuffle_each_iteration=True)

txt.textual_preprocessing(stable_diffusion, placeholder_token_broccoli, placeholder_token_emoji, placeholder_token_combined)

### put all the different components of stable diffusion model into a list
"""all_models = [
    stable_diffusion.text_encoder,
    stable_diffusion.diffusion_model,
    stable_diffusion.decoder,
]"""



### beta is the diffusion rate
noise_scheduler = txt.NoiseScheduler(
    ### beta_start determines the amount of noise added at the start of the denoising process
    beta_start=0.00085,
    ### beta_end at the end of the denoising process
    beta_end=0.012,
    ### the beta_schedule determines that the diffusion rate increases linearly
    beta_schedule="scaled_linear",
    train_timesteps=1000,
)

### Initialize the model we use to fine tune our concept
trainer = txt.StableDiffusionFineTuner(stable_diffusion, noise_scheduler, name="trainer")
#t = txt.StableDiffusionFineTuner(stable_diffusion, noise_scheduler, name="t")


#EPOCHS = 50
### learning rate decays depending on the number of epochs to avoid convergence issues in few epochs 
### in the originial tutorial a scheduler is used but we experienced to have better results without a scheduler
"""learning_rate = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-4, decay_steps=train_ds.cardinality() * EPOCHS
)"""
### inizialize the optimizer
optimizer = tf.keras.optimizers.Adam(
    weight_decay=0.004, learning_rate=1e-4, epsilon=1e-8, global_clipnorm=10
)

trainer.compile(
    optimizer=optimizer,
    # We are performing reduction manually in our train step, so none is required here.
    loss=keras.losses.MeanSquaredError(reduction="none"),
)  

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE
13911/13911 [==============================] - 0s 1us/step


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


136824240/136824240 [==============================] - 1s 0us/step


In [4]:
### EXECUTE THE FOLLOWING TWO BLOCKS ONLY IF YOU WANT TO CONTINUE TRAINING WITH SAVED WEIGTHS
### choose where to load the weights from, either from your google drive or you load our pretrained weights
### make sure to insert the exact name of your weight.npy
path = '/content/drive/MyDrive/1weights_with_two_concepts.npy'

In [ ]:
###load the array of the weights of the text encoder from the training
text_encoder_weights = np.load(path, allow_pickle=True)

### Set the weights of the text encoderstable_diffusion.text_encoder.set_weights(text_encoder_weights)


In [5]:
### training
trainer.fit(
    train_ds,
    epochs=1,
)

220/220 [==============================] - 437s 2s/step - loss: 0.0591


In [6]:
### choose where to save your newly generated weights
path = '/content/drive/MyDrive/weight_with_two_concepts.npy'

In [8]:
###get the weights of the text encoder and save the to Google Drive
text_encoder_weights = np.array(stable_diffusion.text_encoder.get_weights(), dtype=object)

### Save the weights array to a file on your Google Drive
np.save(path, text_encoder_weights)